---
title: "Advanced Bayesian Modelling"
output-file: "advanced_bayesian.html"
format: html
---

In [ ]:
# Setup for Google Colab: Fetch datasets automatically or manually
import os
from google.colab import files

# Define the module and dataset for this notebook
MODULE = '05_advanced'  # e.g., '01_infrastructure'
DATASET = 'large_food_log.csv'  # e.g., 'hippo_diets.csv'
BASE_PATH = '/content/data-analysis-toolkit-FNS'
MODULE_PATH = os.path.join(BASE_PATH, 'notebooks', MODULE)
DATASET_PATH = os.path.join('data', DATASET)

# Step 1: Attempt to clone the repository (automatic method)
# Note: If you encounter a cloning error (e.g., 'fatal: destination path already exists'),
#       reset the runtime (Runtime > Restart runtime) and run this cell again.
try:
    print('Attempting to clone repository...')
    if os.path.exists(BASE_PATH):
        print('Repository already exists, skipping clone.')
    else:
        !git clone https://github.com/ggkuhnle/data-analysis-toolkit-FNS.git
    
    # Debug: Print directory structure
    print('Listing repository contents:')
    !ls {BASE_PATH}
    print(f'Listing notebooks directory contents:')
    !ls {BASE_PATH}/notebooks
    
    # Check if the module directory exists
    if not os.path.exists(MODULE_PATH):
        raise FileNotFoundError(f'Module directory {MODULE_PATH} not found. Check the repository structure.')
    
    # Set working directory to the notebook's folder
    os.chdir(MODULE_PATH)
    
    # Verify dataset is accessible
    if os.path.exists(DATASET_PATH):
        print(f'Dataset found: {DATASET_PATH} 🦛')
    else:
        print(f'Error: Dataset {DATASET} not found after cloning.')
        raise FileNotFoundError
except Exception as e:
    print(f'Cloning failed: {e}')
    print('Falling back to manual upload option...')

    # Step 2: Manual upload option
    print(f'Please upload {DATASET} manually.')
    print(f'1. Click the "Choose Files" button below.')
    print(f'2. Select {DATASET} from your local machine.')
    print(f'3. Ensure the file is placed in notebooks/{MODULE}/data/')
    
    # Create the data directory if it doesn't exist
    os.makedirs('data', exist_ok=True)
    
    # Prompt user to upload the dataset
    uploaded = files.upload()
    
    # Check if the dataset was uploaded
    if DATASET in uploaded:
        with open(DATASET_PATH, 'wb') as f:
            f.write(uploaded[DATASET])
        print(f'Successfully uploaded {DATASET} to {DATASET_PATH} 🦛')
    else:
        raise FileNotFoundError(f'Upload failed. Please ensure you uploaded {DATASET}.')

# Install required packages for this notebook
%pip install pandas numpy
print('Python environment ready.')

# 🧠 5.3 Advanced Bayesian Modelling

This notebook explores advanced Bayesian modelling techniques for nutrition data analysis, building on basic Bayesian methods.

**Objectives**:
- Build hierarchical Bayesian models to account for group variations.
- Interpret posterior distributions for nutrition insights.
- Apply models to `hippo_nutrients.csv` to estimate nutrient intake variations.

**Context**: Hierarchical Bayesian models are powerful for nutrition research, enabling robust estimation of nutrient intakes across groups, such as hippo populations.

<details><summary>Fun Fact</summary>
Bayesian models are like a hippo’s intuition—blending prior knowledge with new data for wise decisions! 🦛
</details>

In [1]:
# Install required packages for Bayesian modelling
%pip install pandas numpy pymc  # Ensures compatibility in Colab
import pandas as pd  # For data manipulation and loading datasets
import numpy as np  # For numerical operations and random sampling
import pymc as pm  # For Bayesian modelling and sampling
print('Bayesian environment ready.')

Bayesian environment ready.


## Data Preparation

Load `hippo_nutrients.csv` from the data handling module and filter for Iron data to model intakes by sex.

In [2]:
# Load the nutrient dataset
df = pd.read_csv('../03_data_handling/data/hippo_nutrients.csv')  # Path relative to advanced module

# Filter for Iron data and remove missing values
df_iron = df[df['Nutrient'] == 'Iron'].dropna()
print(f'Iron data shape: {df_iron.shape}')  # Display number of rows and columns

Iron data shape: (50, 6)


## Hierarchical Bayesian Model

Build a hierarchical Bayesian model to estimate Iron intakes, accounting for differences between female (F) and male (M) hippos.

In [3]:
# Encode Sex as numerical index (F=0, M=1)
sex_idx = df_iron['Sex'].map({'F': 0, 'M': 1}).values

# Define hierarchical model
with pm.Model() as model:
    # Priors for group means (Female and Male)
    mu = pm.Normal('mu', mu=8, sigma=2, shape=2)  # Mean Iron intake for F (0) and M (1)
    sigma = pm.HalfNormal('sigma', sigma=1)  # Shared standard deviation
    
    # Likelihood of observed Iron intakes
    iron = pm.Normal('iron', mu=mu[sex_idx], sigma=sigma, observed=df_iron['Value'])
    
    # Sample from posterior distribution
    trace = pm.sample(1000, tune=1000, return_inferencedata=False)  # 1000 samples after tuning

## Posterior Analysis

Summarise and visualise the posterior distributions of Iron intake means for female and male hippos.

In [4]:
# Calculate posterior means for Female and Male
mu_posterior = trace['mu'].mean(axis=0)
print(f'Posterior means: Female={round(mu_posterior[0], 1)}, Male={round(mu_posterior[1], 1)}')

# Visualise posterior distributions
pm.plot_posterior(trace, var_names=['mu'])  # Plot histograms of mu
plt.show()  # Display plot

Posterior means: Female=8.1, Male=8.0


## Exercise: Build Your Own Model

Create a hierarchical Bayesian model for Calcium intakes, grouped by `Year` (2024, 2025). Summarise the posterior means in a Markdown cell below.

**Guidance**:
- Filter `df` for `Nutrient == 'Calcium'` and remove missing values.
- Encode `Year` as a numerical index (e.g., 2024=0, 2025=1).
- Use `pm.Normal` for group means and `pm.HalfNormal` for sigma.
- Sample and summarise the posterior with `pm.sample()` and `trace['mu'].mean()`.

**Answer**:

My Calcium model code and posterior summary are as follows:

```python
# Your code here
```

**Posterior Summary**:

- Year 2024: Mean = [Your Result]
- Year 2025: Mean = [Your Result]

## Conclusion

You’ve applied advanced Bayesian modelling to estimate nutrient intake variations, capturing uncertainty across groups.

**Next Steps**: Explore database querying with SQL in `5.4_databases_sql.ipynb`.

**Resources**:
- [PyMC Documentation](https://docs.pymc.io/)
- [Bayesian Analysis Guide](https://www.datacamp.com/community/tutorials/bayesian-methods-python)
- Repository: [github.com/ggkuhnle/data-analysis-toolkit-FNS](https://github.com/ggkuhnle/data-analysis-toolkit-FNS)